In [2]:
import pandas as pd
import os

In [3]:
os.listdir()

['.DS_Store',
 '.git',
 '.gitignore',
 'fake_tokens.txt',
 'main.ipynb',
 'miav.ipynb',
 'notebook.ipynb',
 'predictions',
 'README.md',
 'related_work.ipynb',
 'test_trainer',
 'tmp_trainer',
 'words']

## The data

## Reflection on fake news

Fake news has many definitions, ranging from being factually incorrect to misleading, which makes it hard when quantifying results and cross examining results between different studies, since the latent space can be different dependent on the definition of the task. The problem with using objective truth as the definition, that truth can vary depending on culture and context. An actor in a conflict can be seen as the good freedome fighter towards an opressive regime by one side, and as a terrorist on the other side. Lastly, the truth can also change over time, which means that models have to be retrained with up to date information constantly to be able to combat the fake news within the catagory factual correct. Using the definition of misleading would be an inherently easier task since the model would not have to have a world view in order to classify. It should be solveable by using information only present in the text and comparing that to the title. 

## Related work

<!-- Recent studies have shown that fake news propagates through social media at unprecedented speeds. This was observed to happen during the emergence of COVID-19, thus the need to quickly detect and mitigate the spreading of fake news is more important than ever s[1]. 

Many definitions are presented, ranging from being factually incorrect to misleading, and unfortunately, our data source has not specified which definition they use. This makes it harder to interpret why a model predicted as it did, since we do not know if the data contains mostly stories conflicting with reality, or simply written by an overselling journalist. -->

Related work
This paper, written by Shaina Raza & Chen Ding, uses META's BART language model trained on two data sets: NELA-GT-19, which are news articles sourced from multiple sites, and Fakeddit, which is a multimodal dataset from Reddit, consisting of both images and text. The datasets used had more than a binary score, it included labels such as mixed, which is when there is a disagrement whether something is true or false, and categories such as satire into a single category Fake. They discuss their approach of continuously updating the model's training data to retrain the model and stay on top of relevant news. They also assert that freezing a model's weights can quickly make the model outdated since they don't generalize well to future events. Finally, they report an accuracy of 74.89%.



s[1] = https://link.springer.com/article/10.1007/s41060-021-00302-z

s[2] = https://arxiv.org/pdf/2101.00180.pdf